# Wordle solver
by [Laurent Lessard](https://laurentlessard.com)

This is a solver for the popular [Wordle](https://www.powerlanguage.co.uk/wordle/) game.
Run all the cells in order. When you get to "Play the game", follow the instructions to input the game responses.

### Import word lists

In [1]:
function parse_word_list(filename::String)::Vector{String}
    s = open(filename) do file
        read(file, String)
    end
    s = replace(s, '\"' => "")
    return split(s, ", ")
end;

In [2]:
# list of words that can potentially be solutions
solutions = parse_word_list("solutions.txt")

# list of words that are valid guesses, but will never be solutions
nonsolutions = parse_word_list("nonsolutions.txt")

# all possible valid guesses
largelist = [nonsolutions; solutions]
;

### Helper functions

In [3]:
function measure_word(word1::String, word2::String)::Int64
    s = 0
    for i = 1:5
        s *= 10
        if word1[i] == word2[i]
            s += 2
        elseif word1[i] in word2
            s += 1
        else
            s += 0
        end
    end
    return s
end

# return entropy of distribution of group_sizes (we want this to be large!)
function get_entropy(group_sizes::Vector{Int64})::Float64
    pmf = group_sizes/sum(group_sizes)
    return sum( -p*log(p) for p in pmf )
end;

# returns the sizes of the groups of words in `solution_pool` that have the same
# response when the guessed word is `guessed_word`.
function get_group_sizes(
    guessed_word::String,
    solution_pool::Vector{String}
)::Vector{Int64}
    out = Dict{Int64, Int64}()
    for wc in solution_pool
        s = measure_word(guessed_word, wc)
        if haskey(out, s)
            out[s] += 1
        else
            out[s] = 1
        end
    end
    return collect(values(out))
end

Base.Enums.@enum Heuristic begin
    PRIORITIZE_ENTROPY = 1
    PRIORITIZE_MAX_GROUP_SIZE = 2
end

# find next move given a pool of available words
function find_move(
    candidate_pool::Vector{String}, 
    solution_pool::Vector{String};
    heuristic::Heuristic = PRIORITIZE_ENTROPY
)::String
    group_sizes::Vector{Vector{Int64}} = map(w -> get_group_sizes(w, solution_pool), candidate_pool)
    maximum_group_size::Vector{Int64} = map(maximum, group_sizes)
    entropy::Vector{Float64} = map(get_entropy, group_sizes)
    is_potential_solution::Vector{Bool} = map(w -> w in solution_pool, candidate_pool)
    
    if heuristic == PRIORITIZE_ENTROPY
        # first maximize entropy
        # if there are ties, we prefer words in the solution pool
        # if there are still ties, we minimize the maximum group size
        solution_score = zip(entropy, is_potential_solution, -maximum_group_size)
    elseif heuristic == PRIORITIZE_MAX_GROUP_SIZE
        # first minimize the maximum group size
        # if there are ties, we prefer words in the solution pool
        # if there are still ties, we maximize the entropy
        solution_score = zip(-maximum_group_size, is_potential_solution, entropy)
    else
        throw(ArgumentError("Unexpected heuristic."))
    end
    
    # when solution scores are tied, we pick the lexicographically first word
    return maximum(zip(solution_score, candidate_pool))[2]
end

# trim a pool of candidate words based on a current test word and the response it received
function trim_pool(testword::String, response::Int64, pool::Vector{String})
    newpool = [ w for w in pool if measure_word(testword, w) == response ]
    @assert !isempty(newpool) "there are no solutions!"
    return newpool
end;

function trim_pool(testword::String, response::String, pool::Vector{String})
    if length(response) != 5
        println("Your response should be of length 5.")
        return pool
    end
    try
        return trim_pool(testword, parse(Int64, response), pool)
    catch e
        if isa(e, ArgumentError)
            println("Unexpected response; skipping ...")
            return pool
        else
            rethrow(e)
        end
    end
end;

### Find best starting word

In [4]:
# find the best first move according to our heuristic
@time begin
    wfirst = find_move( largelist, solutions )
end

  2.775349 seconds (542.59 k allocations: 117.547 MiB, 3.56% gc time, 2.26% compilation time)


"soare"

# Play the game!
Run each cell in sequence, filling in `response = "....."`
with the response from the wordle website. Use the format `"01020"`, where
- `0` = empty square
- `1` = yellow square
- `2` = green square

If you want to only use more common words as part of your responses, in the line `nextword = find_move(largelist, pool)`, change `largelist` to `solutions`.

In [5]:
pool = solutions
nextword = wfirst
println("FIRST MOVE: ", nextword)

FIRST MOVE: soare


In [6]:
response = "02012"
pool = trim_pool(nextword, response, pool)
println("list of possible solutions ($(length(pool))): ", join(pool, ", "))
nextword = find_move(largelist, pool)
println("NEXT MOVE: ", nextword)

list of possible solutions (9): forge, rogue, rouge, gorge, horde, route, force, borne, forte
NEXT MOVE: grunt


In [7]:
response = "21000"
pool = trim_pool(nextword, response, pool)
println("list of possible solutions ($(length(pool))): ", join(pool, ", "))
nextword = find_move(largelist, pool)
println("NEXT MOVE: ", nextword)

list of possible solutions (1): gorge
NEXT MOVE: gorge


In [8]:
response = "....."
pool = trim_pool(nextword, response, pool)
println("list of possible solutions ($(length(pool))): ", join(pool, ", "))
nextword = find_move(largelist, pool)
println("NEXT MOVE: ", nextword)

Unexpected response; skipping ...
list of possible solutions (1): gorge
NEXT MOVE: gorge


In [9]:
response = "....."
pool = trim_pool(nextword, response, pool)
println("list of possible solutions ($(length(pool))): ", join(pool, ", "))
nextword = find_move(largelist, pool)
println("NEXT MOVE: ", nextword)

Unexpected response; skipping ...
list of possible solutions (1): gorge
NEXT MOVE: gorge
